In [6]:
!pip install chainlit
!pip install ctransformers
!pip install torch
!pip install sentence_transformers
!pip install chromadb
!pip install langchain
!pip install PyMuPDF
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 6.8 MB/s eta 0:00:00


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader

In [7]:
import os
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

#path to your document 
loader = PyPDFLoader("/content/Topic 6 Notes.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

vector_store = Chroma.from_documents(texts, embeddings, collection_metadata={"hnsw:space": "cosine"}, persist_directory="stores/pet_cosine")

In [8]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import CTransformers
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from io import BytesIO
from langchain.document_loaders import PyPDFLoader

downloading the model from huggingface

In [9]:
!wget https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q5_K_S.gguf?download=true

--2023-11-27 20:28:25--  https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q5_K_S.gguf?download=true
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/fe/17/fe17596731f84a0d03bece77489780bc7e068323c0aeca88b6393d3e9e65dd49/a47ebae2072b4c330baa36365a76d49732ed67191ffa079a14478a59e9358076?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27zephyr-7b-beta.Q5_K_S.gguf%3B+filename%3D%22zephyr-7b-beta.Q5_K_S.gguf%22%3B&Expires=1701376105&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMTM3NjEwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZlLzE3L2ZlMTc1OTY3MzFmODRhMGQwM2JlY2U3NzQ4OTc4MGJjN2UwNjgzMjNjMGFlY2E4OGI2MzkzZDNlOWU2NWRkNDkvYTQ3ZWJhZTIwNzJiNGMzMzBiYWEz

setting up the prompt template

In [10]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [11]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
load_vector_store = Chroma(persist_directory="stores/chunk_data", embedding_function=embeddings)
retriever = load_vector_store.as_retriever(search_kwargs={"k":3})
query = "what is a pseudo code?"
semantic_search = retriever.get_relevant_documents(query)
print(semantic_search)

[Document(page_content='Unlike structure English, Pseudo -code mostly resembles programming language. All descriptions \nand comments are included in Pseudo -code.  \nThe constructs of some of the programming languages like C, Fortran, Pascal are used for writing \nPseudo -code.  \nThe Pseudo code language is computer understandable lan guage which provides the details and \nmethodology of performing the task.  \nExample  \nProgram to print Fibonacci up to n numbers.   \nvoid function Fibonacci  \nGet value of n;  \nSet value of a to 1;  \nSet value of b to 1;  \nInitialize I to 0  \nfor (i=0; i< n; i++)  \n{ \nif a greater than b', metadata={'page': 6, 'source': '/content/Topic 6 Notes.pdf'}), Document(page_content='Structured programming uses some of the tokens such as :  \n \nIF-THEN -ELSE,   \nDO-WHILE -UNTIL  \n \nThe variables and tokens are stored in th e Data Dictionary. The use of the stored variables and \ntokens enables the analyst to appropriately understand and thus make t

In [22]:
from langchain.llms import ctransformers
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from io import BytesIO
from langchain.document_loaders import PyPDFLoader

In [33]:
local_llm = "zephyr-7b-beta.Q5_K_S.gguf"

config = {
    'max_new_tokens': 1024,
    'repetition_penalty': 1.1,
    'temperature': 0.1,
    'top_k': 50,
    'top_p': 0.9,
    'stream': True,
    'device': 'cuda:0'  # Specify the GPU device to use
}

llm = CTransformers(
    model=local_llm,
    model_type="mistral",
    **config
)

In [34]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [35]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
load_vector_store = Chroma(persist_directory="stores/chunk_data", embedding_function=embeddings)
retriever = load_vector_store.as_retriever(search_kwargs={"k":1})
chain_type_kwargs = {"prompt": prompt}

In [36]:
def get_response(input):
  query = input
  chain_type_kwargs = {"prompt": prompt}
  qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)
  response = qa(query)
  return response

In [37]:
get_response("what are the ways of data processing")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what are the ways of data processing',
 'result': 'Data processing can be done from two perspectives - logically, as viewed by the user, or physically, as executed by the software. The sources and destinations of the data involved in these processes are stored in the Data Store, which includes both internal and external files, as well as tables with naming conventions and indexing properties.',
 'source_documents': [Document(page_content='• Content Description (Notation etc. )  \n• Supplementary Information (preset values, constraints etc.)  \nData Store  \nThe information related to the sources of data flow and the destinations of the data is stored in \nData Store. Data Store comprise of the following - \n• Files  \no Internal to software.  \no External to software on the same machine.  \no External to software and system, located on different machine.  \n• Tables  \no Naming convention  \no Indexing property  \nData Processing  \nProcessing of data can be done in two diff